In [1]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 114.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 46.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.1 MB/s eta 0:00:00


In [2]:
!pip install datasets

In [3]:
!pip install evaluate

In [4]:
!pip install rouge_score

In [20]:
import os
import torch
import pandas as pd
from datasets import *
from sklearn.model_selection import train_test_split
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
    AutoModel
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [4]:
# The model that you want to train from the Hugging Face hub
model_name = "NousResearch/Llama-2-7b-chat-hf"

# The instruction dataset to use
dataset_name = "cnn_dailymail"

# Fine-tuned model name
new_model = "llama-2-7b-miniguanaco"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule (constant a bit better than cosine)
lr_scheduler_type = "constant"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 25

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [5]:
df_train=load_dataset(dataset_name,'3.0.0',split='train[:800]')
df_test=load_dataset(dataset_name,'3.0.0',split='test[:100]')
len(df_test)

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

100

In [6]:
df_train

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 800
})

In [10]:
words=[len(x.split(" ")) for x in df_train['article']]
sum(words)/len(words)

598.76375

In [7]:
df1=pd.DataFrame({"text":["<s>[INST] Summarize the question based on the context below. " + "\n[context]: " + df_train['article'][i] + "[\INST] "+df_train['highlights'][i] for i in range(len(df_train)) if len(df_train['article'][i].split(" "))<700]})
df2=pd.DataFrame({"text":["<s>[INST] Summarize the question based on the context below. " + "\n[context]: " + df_test['article'][i] + "[\INST] "+df_test['highlights'][i] for i in range(len(df_test)) if len(df_test['article'][i].split(" "))<700]})
df1

,text
0,<s>[INST] Summarize the question based on the ...
1,<s>[INST] Summarize the question based on the ...
2,<s>[INST] Summarize the question based on the ...
3,<s>[INST] Summarize the question based on the ...
4,<s>[INST] Summarize the question based on the ...
...,...
532,<s>[INST] Summarize the question based on the ...
533,<s>[INST] Summarize the question based on the ...
534,<s>[INST] Summarize the question based on the ...
535,<s>[INST] Summarize the question based on the ...


In [8]:
print(df1.iloc[1]['text'])

<s>[INST] Summarize the question based on the context below. 
[context]: WASHINGTON (CNN) -- Doctors removed five small polyps from President Bush's colon on Saturday, and "none appeared worrisome," a White House spokesman said. The polyps were removed and sent to the National Naval Medical Center in Bethesda, Maryland, for routine microscopic examination, spokesman Scott Stanzel said. Results are expected in two to three days. All were small, less than a centimeter [half an inch] in diameter, he said. Bush is in good humor, Stanzel said, and will resume his activities at Camp David. During the procedure Vice President Dick Cheney assumed presidential power. Bush reclaimed presidential power at 9:21 a.m. after about two hours. Doctors used "monitored anesthesia care," Stanzel said, so the president was asleep, but not as deeply unconscious as with a true general anesthetic. He spoke to first lady Laura Bush -- who is in Midland, Texas, celebrating her mother's birthday -- before and af

In [9]:
df2

,text
0,<s>[INST] Summarize the question based on the ...
1,<s>[INST] Summarize the question based on the ...
2,<s>[INST] Summarize the question based on the ...
3,<s>[INST] Summarize the question based on the ...
4,<s>[INST] Summarize the question based on the ...
...,...
64,<s>[INST] Summarize the question based on the ...
65,<s>[INST] Summarize the question based on the ...
66,<s>[INST] Summarize the question based on the ...
67,<s>[INST] Summarize the question based on the ...


In [10]:
train_ds=Dataset.from_pandas(df1)
test_ds=Dataset.from_pandas(df2)
ds=DatasetDict()
ds['train']=train_ds
ds['test']=test_ds
ds

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 537
    })
    test: Dataset({
        features: ['text'],
        num_rows: 69
    })
})

In [11]:
df_train_for_gen=pd.DataFrame({"text":["<s>[INST] Summarize the question based on the context below. " + "\n[context]: " + df_train['article'][i] + "[\INST] " for i in range(len(df_train)) if len(df_train['article'][i].split(" "))<700]})
df_test_for_gen=pd.DataFrame({"text":["<s>[INST] Summarize the question based on the context below. " + "\n[context]: " + df_test['article'][i] + "[\INST] " for i in range(len(df_test)) if len(df_test['article'][i].split(" "))<700]})

In [12]:
df_train_for_gen

,text
0,<s>[INST] Summarize the question based on the ...
1,<s>[INST] Summarize the question based on the ...
2,<s>[INST] Summarize the question based on the ...
3,<s>[INST] Summarize the question based on the ...
4,<s>[INST] Summarize the question based on the ...
...,...
532,<s>[INST] Summarize the question based on the ...
533,<s>[INST] Summarize the question based on the ...
534,<s>[INST] Summarize the question based on the ...
535,<s>[INST] Summarize the question based on the ...


In [13]:
df_test_for_gen

,text
0,<s>[INST] Summarize the question based on the ...
1,<s>[INST] Summarize the question based on the ...
2,<s>[INST] Summarize the question based on the ...
3,<s>[INST] Summarize the question based on the ...
4,<s>[INST] Summarize the question based on the ...
...,...
64,<s>[INST] Summarize the question based on the ...
65,<s>[INST] Summarize the question based on the ...
66,<s>[INST] Summarize the question based on the ...
67,<s>[INST] Summarize the question based on the ...


In [14]:
# The model that you want to train from the Hugging Face hub
model_name = "NousResearch/Llama-2-7b-chat-hf"

# Fine-tuned model name
new_model = "llama-2-7b-miniguanaco"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule (constant a bit better than cosine)
lr_scheduler_type = "constant"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 25

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [15]:
# Load dataset (you can process it here)
dataset = ds['train']

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=3,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

import torch
import torch.optim as optim
from transformers import Trainer, TrainingArguments

def train_rl_policy(model, reward_model):
    # Define the optimizer
    optimizer = optim.Adam(model.parameters())

    # Define the training arguments
    training_args = TrainingArguments(
        output_dir="./output",
        num_train_epochs=3,
        per_device_train_batch_size=16,
        save_steps=1000,
        evaluation_strategy="steps",
        eval_steps=1000,
    )

    # Define the trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        compute_metrics=lambda _pt_outputs: {"reward": reward_model(_pt_outputs["logits"])},
    )

    # Train the policy
    trainer.train()

    return policy



# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained(new_model)

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
25,1.927200
50,1.683900
75,1.612100
100,1.599400
125,1.643400
150,1.605800
175,1.520400
200,1.557200
225,1.623000
250,1.565200


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

Step,Training Loss
25,1.927200
50,1.683900
75,1.612100
100,1.599400
125,1.643400
150,1.605800
175,1.520400
200,1.557200
225,1.623000
250,1.565200


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

# Code for validating


```
trainer = SFTTrainer(
    ...
    ...
)

epoch_val_loss = 0.0
for epoch in range(num_train_epochs):
    trainer.train()

    val_loss = evaluate(trainer.model, val_dataset)
    epoch_val_loss += val_loss

    print(f"Epoch {epoch + 1}/{num_train_epochs}: Validation Loss = {val_loss}")

avg_val_loss = epoch_val_loss / num_train_epochs
print(f"Average Validation Loss: {avg_val_loss}")

```


In [16]:
df_test_for_gen.iloc[3]['text']

'<s>[INST] Summarize the question based on the context below. \n[context]: (CNN)A Duke student has admitted to hanging a noose made of rope from a tree near a student union, university officials said Thursday. The prestigious private school didn\'t identify the student, citing federal privacy laws. In a news release, it said the student was no longer on campus and will face student conduct review. The student was identified during an investigation by campus police and the office of student affairs and admitted to placing the noose on the tree early Wednesday, the university said. Officials are still trying to determine if other people were involved. Criminal investigations into the incident are ongoing as well. Students and faculty members marched Wednesday afternoon chanting "We are not afraid. We stand together,"  after pictures of the noose were passed around on social media. At a forum held on the steps of Duke Chapel, close to where the noose was discovered at 2 a.m., hundreds of 

In [31]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)
count=0
for i in range(len(df_train_for_gen[:2])):
# Run text generation pipeline with our next model
  prompt =df_train_for_gen.iloc[i]['text']
  pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_new_tokens=300)
  result = pipe(prompt)
#print(result[0]['generated_text'])
  df1.at[i,"predicted_summary"]=result
  count+=1
  print(count)
  break

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


1


In [40]:
df1.iloc[0]['predicted_summary'][0]['generated_text'].split("[\INST]")[0].lstrip("<s>[INST] Summarize the question based on the context below. \n[context]:")

'LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don\'t think I\'ll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office chart. Details o

In [41]:
df1.iloc[0]['predicted_summary'][0]['generated_text'].split("[\INST]")[1]

'  The question in this article is: How will Harry Potter star Daniel Radcliffe celebrate his 18th birthday?]  Based on the article, it seems that Daniel Radcliffe does not have any grand plans to celebrate his 18th birthday, despite having gained access to a reported £20 million fortune. He has stated that he does not plan to be extravagant or buy expensive things, and instead plans to spend his money on things that cost around £10, such as books, CDs, and DVDs. He also mentioned that he will have some sort of party, but did not provide any details. Therefore, the answer to the question is: Daniel Radcliffe does not have any specific plans to celebrate his 18th birthday, and is instead focusing on keeping his feet firmly on the ground and avoiding the temptation to spend his money extravagantly.'

In [22]:
from peft import LoraConfig, get_peft_model

In [23]:
lora_config = LoraConfig.from_pretrained('Ritirupa/llama2_reward_test')
reward_model = get_peft_model(model, lora_config)

In [44]:
prompts=[]
passage=df1.iloc[0]['predicted_summary'][0]['generated_text'].split("[\INST]")[0].lstrip("<s>[INST] Summarize the question based on the context below. \n[context]:")
summary=df1.iloc[0]['predicted_summary'][0]['generated_text'].split("[\INST]")[1]
prompt_for_finetune = f'''Analyze the highlight written and the passage based on which the
highlight was supposed to be written and then relate the scores provided below.
Passage:
    {passage}

    SUmmary:
    {summary}

    Prediction Task:
    Classify the summary as 1(good) or 0(bad). Limit your answer to 1 or 0.'''
prompts.append(prompt_for_finetune)

df1.at[0,'prompt_for_finetune'] = prompts[0]

In [45]:
df1.iloc[0]['prompt_for_finetune']

'Analyze the highlight written and the passage based on which the \nhighlight was supposed to be written and then relate the scores provided below.\nPassage:\n    LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don\'t think I\'ll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble

In [49]:
device = "cuda:0"
inputs = tokenizer(df1.iloc[0]['prompt_for_finetune'],
                   return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=12)
print(tokenizer.decode(outputs[0], skip_special_tokens=False))

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


<s> Analyze the highlight written and the passage based on which the 
highlight was supposed to be written and then relate the scores provided below.
Passage:
    LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don't plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don't think I'll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in 

In [37]:
print(len(df_train_for_gen.iloc[7]['text']))
prompt =df_train_for_gen.iloc[7]['text']
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=700)
result = pipe(prompt)
result

1551


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


[{'generated_text': '<s>[INST] Summarize the question based on the context below. \n[context]: WASHINGTON (CNN) -- Vice President Dick Cheney will serve as acting president briefly Saturday while President Bush is anesthetized for a routine colonoscopy, White House spokesman Tony Snow said Friday. Bush is scheduled to have the medical procedure, expected to take about 2 1/2 hours, at the presidential retreat at Camp David, Maryland, Snow said. Bush\'s last colonoscopy was in June 2002, and no abnormalities were found, Snow said. The president\'s doctor had recommended a repeat procedure in about five years. The procedure will be supervised by Dr. Richard Tubb and conducted by a multidisciplinary team from the National Naval Medical Center in Bethesda, Maryland, Snow said. A colonoscopy is the most sensitive test for colon cancer, rectal cancer and polyps, small clumps of cells that can become cancerous, according to the Mayo Clinic. Small polyps may be removed during the procedure. Sno

In [19]:
df1['predicted_summary'][0]

[{'generated_text': '<s>[INST] Summarize the question based on the context below. \n[context]: LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don\'t think I\'ll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: P

In [27]:
result[0]['generated_text'].split("[\INST]")[1]

"  Vice President Dick Cheney will serve as acting president briefly Saturday.\nBush is scheduled to have a routine colonoscopy at Camp David, Maryland.\nBush's last colonoscopy was in June 2002, and no abnormalities were found.\nA colonoscopy is the most sensitive test for colon cancer, rectal cancer.\nSnow said that was the case when Bush had colonoscopies before becoming president.\nSnow himself is undergoing chemotherapy for cancer that began in his colon.\nSnow told reporters he had a chemo session scheduled later Friday.\nThe American Cancer Society recommends that people without high-risk factors or symptoms begin getting screened for signs of colorectal cancer at age 50.\nE-mail to a friend."

In [ ]:
df2.iloc[3]['text'].split("[\INST]")[1]

' 17 Americans were exposed to the Ebola virus while in Sierra Leone in March .\nAnother person was diagnosed with the disease and taken to hospital in Maryland .\nNational Institutes of Health says the patient is in fair condition after weeks of treatment .'

In [38]:
from tqdm import tqdm

tqdm.pandas()

def generate_summary(text):
  if(len(df_train_for_gen.iloc[1]['text'])<2000):
    logging.set_verbosity(logging.CRITICAL)
    prompt =text
    pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=600)
    result = pipe(prompt)
  else:
    logging.set_verbosity(logging.CRITICAL)
    prompt =text
    pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=800)
    result = pipe(prompt)
  return(result)

df1['predicted_summary']=df_train_for_gen['text'].progress_apply(lambda x: generate_summary(x))

  0%|          | 0/537 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
  1%|▏         | 7/537 [15:45<19:52:59, 135.06s/it]


KeyboardInterrupt: ignored

In [39]:
model=AutoModelForCausalLM.from_pretrained('/content/llama-2-7b-miniguanaco/adapter_model.bin')

OSError: ignored

In [30]:
df1['predicted_summary'][1]

KeyError: ignored

In [ ]:
print(generate_summary(df_train_for_gen['text'][1]))

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


[{'generated_text': '<s>[INST] Summarize the question based on the context below. \n[context]: Editor\'s note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events. Here, Soledad O\'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial. MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor." Here, inmates with the most severe mental illnesses are incarcerated until they\'re ready to appear in court. Most often, they face drug charges or charges of assaulting an officer --charges that Judge Steven Leifman says are usually "avoidable felonies." He says the arrests often result from confrontations with police. Mentally ill people often won\'t do what they\'re told when police arrive on the scene -- confrontation seems 

In [ ]:
df2['predicted_summary']=df_test_for_gen['text'].progress_apply(lambda x: generate_summary(x))

  0%|          | 0/50 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
100%|██████████| 50/50 [06:30<00:00,  7.81s/it]


In [27]:
from tqdm import tqdm

tqdm.pandas()
df1['reference_summary']=df1['text'].progress_apply(lambda x: x.split("[\INST]")[1])
df2['reference_summary']=df2['text'].progress_apply(lambda x: x.split("[\INST]")[1])

100%|██████████| 50/50 [00:00<00:00, 45570.45it/s]


In [28]:
df1

,text,predicted_summary,reference_summary
0,<s>[INST] Summarize the question based on the ...,[{'generated_text': '<s>[INST] Summarize the q...,Harry Potter star Daniel Radcliffe gets £20M ...
1,<s>[INST] Summarize the question based on the ...,[{'generated_text': '<s>[INST] Summarize the q...,Mentally ill inmates in Miami are housed on t...
2,<s>[INST] Summarize the question based on the ...,[{'generated_text': '<s>[INST] Summarize the q...,"NEW: ""I thought I was going to die,"" driver s..."
3,<s>[INST] Summarize the question based on the ...,[{'generated_text': '<s>[INST] Summarize the q...,"Five small polyps found during procedure; ""no..."
4,<s>[INST] Summarize the question based on the ...,[{'generated_text': '<s>[INST] Summarize the q...,"NEW: NFL chief, Atlanta Falcons owner critica..."
...,...,...,...
545,<s>[INST] Summarize the question based on the ...,NaN,Chilean judge orders prosecution of 98 former...
546,<s>[INST] Summarize the question based on the ...,NaN,NEW: Sen. Obama says Cuba policy will be base...
547,<s>[INST] Summarize the question based on the ...,NaN,Valentino Rossi claims pole position for Sund...
548,<s>[INST] Summarize the question based on the ...,NaN,Sharon Stone had suggested that deadly earthq...


In [ ]:
df1.to_csv("train_generated_summary.csv")
df2.to_csv("test_generated_summary.csv")

In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=8026dbe56e4616a5123179db19547fb436ed10e4b403f3996634efcdaf2849e9
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
df1=pd.read_csv('/content/train_generated_summary.csv')
df2=pd.read_csv('/content/test_generated_summary.csv')
df1['predicted_summary'][1]

' 9'

In [35]:
import evaluate

# Load the ROUGE evaluation metric
rouge = evaluate.load('rouge')

# Define the candidate predictions and reference sentences
predictions = [result[0]['generated_text'].split("[\INST]")[1]]
references = [df1.iloc[1]['text'].split("[\INST]")[1]]

# Compute the ROUGE score
results = rouge.compute(predictions=predictions, references=references)

# Print the results
print(results)

{'rouge1': 0.21487603305785125, 'rouge2': 0.05042016806722689, 'rougeL': 0.1487603305785124, 'rougeLsum': 0.18181818181818182}


In [36]:
references

[' Five small polyps found during procedure; "none worrisome," spokesman says .\nPresident reclaims powers transferred to vice president .\nBush undergoes routine colonoscopy at Camp David .']

In [34]:
df2.iloc[0]['text'].split("[\INST]")[1]

' Membership gives the ICC jurisdiction over alleged crimes committed in Palestinian territories since last June .\nIsrael and the United States opposed the move, which could open the door to war crimes investigations against Israelis .'